***
# **Libraries**
***
Import needed common libraries

In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql

import os
print(os.listdir("../BD00-Datasets"))

['csv-datasets', 'db-datasets', 'New folder', '__MACOSX']


***
# **Data retrieval**
***
Construct helper function for data connection and queries

In [3]:
#conn = sqlite3.connect('dataset/Northwind_large.sqlite')
#c = conn.cursor()
def run_query(q):
    with sql.connect("../BD00-Datasets/db-datasets/Northwind.db") as conn:
        return pd.read_sql_query(q, conn)

***
# **Data inspections**
***

View data structures

Construct needed data into dataset

Convert data types

In [4]:
# Inspecting data tables
#run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables = run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables

,name,type
0,Categories,table
1,Customers,table
2,Employees,table
3,Orders,table
4,OrderDetails,table
5,Products,table
6,Regions,table
7,Shippers,table
8,Suppliers,table
9,Territories,table


In [19]:
q = '''SELECT od.OrderId, od.ProductId, p.ProductName, od.Quantity, od.UnitPrice, 
              (od.Quantity * od.UnitPrice) AS OrderValue, od.Discount,
              ((od.Quantity * od.UnitPrice) - ((od.Quantity * od.UnitPrice) * od.Discount)) AS TotalValue,
              o.OrderDate, o.CustomerId, c.ContactName AS CustomerName,
              o.ShipCountry AS Country 
       FROM OrderDetails AS od
       JOIN Products AS p ON od.ProductId = p.Id
       JOIN Orders AS o ON od.OrderId = o.Id
       JOIN Customers AS c ON o.CustomerId = c.Id
    '''

orders = run_query(q)
orders.head()

,OrderId,ProductId,ProductName,Quantity,UnitPrice,OrderValue,Discount,TotalValue,OrderDate,CustomerId,CustomerName,Country
0,10248,11,Queso Cabrales,12,14.0,168.0,0.0,168.0,2012-07-04,VINET,Paul Henriot,France
1,10248,42,Singaporean Hokkien Fried Mee,10,9.8,98.0,0.0,98.0,2012-07-04,VINET,Paul Henriot,France
2,10248,72,Mozzarella di Giovanni,5,34.8,174.0,0.0,174.0,2012-07-04,VINET,Paul Henriot,France
3,10249,14,Tofu,9,18.6,167.4,0.0,167.4,2012-07-05,TOMSP,Karin Josephs,Germany
4,10249,51,Manjimup Dried Apples,40,42.4,1696.0,0.0,1696.0,2012-07-05,TOMSP,Karin Josephs,Germany


In [20]:
# Import datetime library
from datetime import datetime, timedelta

In [21]:
orders["OrderDate"] = pd.to_datetime(orders["OrderDate"])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621806 entries, 0 to 621805
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   OrderId       621806 non-null  int64         
 1   ProductId     621806 non-null  int64         
 2   ProductName   621806 non-null  object        
 3   Quantity      621806 non-null  int64         
 4   UnitPrice     621806 non-null  float64       
 5   OrderValue    621806 non-null  float64       
 6   Discount      621806 non-null  float64       
 7   TotalValue    621806 non-null  float64       
 8   OrderDate     621806 non-null  datetime64[ns]
 9   CustomerId    621806 non-null  object        
 10  CustomerName  621806 non-null  object        
 11  Country       621806 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 56.9+ MB


***
# **Aggregate data at customer level**
***

Recency: today date - last order date by each customer 

Frequency: the count of unique order/invoice by each customer

Monetary: the total amount (sum) of paid expenses by each customer

In [22]:
today = orders['OrderDate'].max().date() + timedelta(days=5)
#df_customer = orders.groupby('CustomerId').agg({'OrderDate': lambda x: (today - x.max().date()).days,                          #                                                  'OrderId': lambda x: len(x.unique()),  
#                                               'TotalValue': lambda x: round(x.sum(),2)})
#df_customer.columns = ['Recency', 'Frequency', 'Monetary']
#df_customer.sort_values('Recency', ascending=False)

df_customer = pd.DataFrame(orders.pivot_table(
                           index="CustomerId", 
                           values=["OrderDate", "OrderId", "TotalValue"], 
                           aggfunc={"OrderDate": lambda x: (today - x.max().date()).days,
                                      "OrderId": pd.Series.nunique, 
                                   "TotalValue": lambda x: round(x.sum(),2)}))

df_customer.columns = ["Recency", "Frequency", "Monetary"]
df_customer.sort_values('Recency', ascending=False)

,Recency,Frequency,Monetary
CustomerId,,,
RICSU,45,195,5094316.46
SAVEA,33,218,5394854.75
WOLZA,31,187,5196525.31
BSBEV,30,193,4908283.21
HUNGC,29,197,5410736.54
...,...,...,...
VICTE,5,182,4725721.66
VINET,5,191,5584238.62
ROMEY,5,185,5265845.64


***
# **Recency, Frequency, and Monetary score**
***

Segregate Recency, Frequency, and Monetary values into discrete values
and classify them into categorical order

In [23]:
def RFM_Quartile (df, rfm, rev=False):
    if (rev == True):
      lbl = [4, 3, 2, 1]
    else: lbl = [1, 2, 3, 4]

    return (pd.cut(df[rfm], 
                   bins=[-1, np.percentile(df[rfm], 25), 
                             np.percentile(df[rfm], 50), 
                             np.percentile(df[rfm], 75), 
                             df[rfm].max()], 
                   labels=lbl).astype("int"))      

In [24]:
# Recency score: 
# Days since last purchase: 1 (17++ days) || 2 (11-16 days) || 3 (8-10 days) || 4 (within a week)
df_customer['R_Score'] = RFM_Quartile(df_customer, "Recency", True )
                                    
# Frequency score: 
# Total number of orders: 1 (177--) || 2 (178-183) || 3 (184-193) || 4 (194++)
df_customer["F_Score"] = RFM_Quartile(df_customer, "Frequency")

# Monetary score: 
# Total paid expenses: 1 (4761k--) || 2 (4782k-5055k) || 3 (5069k-5269k) || 4 (5277k++)
df_customer["M_Score"] = RFM_Quartile(df_customer, "Monetary")

df_customer.head().sort_values('Monetary', ascending=False)

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score
CustomerId,,,,,,
BERGS,6,206,5675614.51,4,4,4
ALFKI,7,200,5317193.08,4,4,4
ANTON,6,191,5262405.09,4,3,3
ANATR,7,192,5208686.42,4,3,3
AROUT,15,162,4395220.43,2,1,1


***
# **RFM score**
***

The combined (concatenated) score of Recency, Frequency, and Monetary scores

In [25]:
#df_customer["RFM_Score"] = ((df_customer["R_Score"].map(str)) + 
                            #(df_customer["F_Score"].map(str)) + 
                             #df_customer["M_Score"].map(str))
df_customer["RFM_Score"] = ((df_customer["R_Score"] * 100) + 
                            (df_customer["F_Score"] * 10) + 
                             df_customer["M_Score"])

df_customer.head().sort_values('RFM_Score', ascending=False)

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
CustomerId,,,,,,,
ALFKI,7,200,5317193.08,4,4,4,444
BERGS,6,206,5675614.51,4,4,4,444
ANATR,7,192,5208686.42,4,3,3,433
ANTON,6,191,5262405.09,4,3,3,433
AROUT,15,162,4395220.43,2,1,1,211


***
# **Labeling**
***
Labeling based on RFM scores

In [27]:
#conditions = [
# "Premium Customers" ----------------------------------------------
           # (df_customer["RFM_Score"] == '444'), 
# "Loyal Customers" ------------------------------------------------
           # (df_customer["RFM_Score"] == '443') 
               # | (df_customer["RFM_Score"] == '434') 
                   # | (df_customer["RFM_Score"] == '433') |
           # (df_customer["RFM_Score"] == '344') 
               # | (df_customer["RFM_Score"] == '343') 
                   # | (df_customer["RFM_Score"] == '334'), 
# "Frequent Buyers" -----------------------------------------------
           # (df_customer["RFM_Score"] == '442') 
               # | (df_customer["RFM_Score"] == '441')
                   # | (df_customer["RFM_Score"] == '342') 
                       # | (df_customer["RFM_Score"] == '341') | 
           # (df_customer["RFM_Score"] == '432') 
               # | (df_customer["RFM_Score"] == '431')
                   # | (df_customer["RFM_Score"] == '333')
                       # | (df_customer["RFM_Score"] == '332') 
                           # | (df_customer["RFM_Score"] == '331'),
# "Big Spenders" --------------------------------------------------
           # (df_customer["RFM_Score"] == '424') 
               # | (df_customer["RFM_Score"] == '414') 
                   # | (df_customer["RFM_Score"] == '324') 
                       # | (df_customer["RFM_Score"] == '314') |
           # (df_customer["RFM_Score"] == '423') 
               # | (df_customer["RFM_Score"] == '413') 
                   # | (df_customer["RFM_Score"] == '323') 
                       # | (df_customer["RFM_Score"] == '313'), 
# "Promising New Customers" ---------------------------------------
           # (df_customer["RFM_Score"] == '422') 
               # | (df_customer["RFM_Score"] == '421')
                   # | (df_customer["RFM_Score"] == '412') 
                       # | (df_customer["RFM_Score"] == '411') |
           # (df_customer["RFM_Score"] == '322') 
               # | (df_customer["RFM_Score"] == '321')
                   # | (df_customer["RFM_Score"] == '312') 
                       # | (df_customer["RFM_Score"] == '311'),
# "Inactive Frequent Buyers" --------------------------------------
           # (df_customer["RFM_Score"] == '244') 
               # | (df_customer["RFM_Score"] == '243') 
                   # | (df_customer["RFM_Score"] == '242') 
                       # | (df_customer["RFM_Score"] == '241') |
           # (df_customer["RFM_Score"] == '144') 
               # | (df_customer["RFM_Score"] == '143') 
                   # | (df_customer["RFM_Score"] == '142') 
                       # | (df_customer["RFM_Score"] == '141') |
           # (df_customer["RFM_Score"] == '232') 
               # | (df_customer["RFM_Score"] == '231')
                   # | (df_customer["RFM_Score"] == '132') 
                       # | (df_customer["RFM_Score"] == '131'),
# "Inactive High Value" -------------------------------------------
           # (df_customer["RFM_Score"] == '234') 
               # | (df_customer["RFM_Score"] == '224') 
                   # | (df_customer["RFM_Score"] == '214') 
                       # | (df_customer["RFM_Score"] == '134') 
                           # | (df_customer["RFM_Score"] == '124') |
           # (df_customer["RFM_Score"] == '233')
               # | (df_customer["RFM_Score"] == '223') 
                   # | (df_customer["RFM_Score"] == '213')
                       # | (df_customer["RFM_Score"] == '133')
                           # | (df_customer["RFM_Score"] == '123'), 
# "Inactive Customers" --------------------------------------------
           # (df_customer["RFM_Score"] == '222') 
               # | (df_customer["RFM_Score"] == '221')
                   # | (df_customer["RFM_Score"] == '212')
                       # | (df_customer["RFM_Score"] == '211'),
# "Lost Customers" ------------------------------------------------
           # (df_customer["RFM_Score"] == '122') 
               # | (df_customer["RFM_Score"] == '121')
                   # | (df_customer["RFM_Score"] == '114') 
                       # | (df_customer["RFM_Score"] == '113') 
                           # | (df_customer["RFM_Score"] == '112')
# "Lost Cheap Customers" ------------------------------------------
           # | (df_customer["RFM_Score"] == 111)
#]

In [28]:
conditions = [
# "Premium Customers" ----------------------------------------------
            (df_customer["RFM_Score"] == 444), 
# "Loyal Customers" ------------------------------------------------
            (df_customer["RFM_Score"] == 443) 
                | (df_customer["RFM_Score"] == 434) 
                    | (df_customer["RFM_Score"] == 433) |
            (df_customer["RFM_Score"] == 344) 
                | (df_customer["RFM_Score"] == 343) 
                    | (df_customer["RFM_Score"] == 334), 
# "Frequent Buyers" -----------------------------------------------
            (df_customer["RFM_Score"] == 442) 
                | (df_customer["RFM_Score"] == 441)
                    | (df_customer["RFM_Score"] == 342) 
                        | (df_customer["RFM_Score"] == 341) | 
            (df_customer["RFM_Score"] == 432) 
                | (df_customer["RFM_Score"] == 431)
                    | (df_customer["RFM_Score"] == 333)
                        | (df_customer["RFM_Score"] == 332) 
                            | (df_customer["RFM_Score"] == 331),
# "Big Spenders" --------------------------------------------------
            (df_customer["RFM_Score"] == 424) 
                | (df_customer["RFM_Score"] == 414) 
                    | (df_customer["RFM_Score"] == 324) 
                        | (df_customer["RFM_Score"] == 314) |
            (df_customer["RFM_Score"] == 423) 
                | (df_customer["RFM_Score"] == 413) 
                    | (df_customer["RFM_Score"] == 323) 
                        | (df_customer["RFM_Score"] == 313), 
# "Promising New Customers" ---------------------------------------
            (df_customer["RFM_Score"] == 422) 
                | (df_customer["RFM_Score"] == 421)
                    | (df_customer["RFM_Score"] == 412) 
                        | (df_customer["RFM_Score"] == 411) |
            (df_customer["RFM_Score"] == 322) 
                | (df_customer["RFM_Score"] == 321)
                    | (df_customer["RFM_Score"] == 312) 
                        | (df_customer["RFM_Score"] == 311),
# "Inactive Frequent Buyers" --------------------------------------
            (df_customer["RFM_Score"] == 244) 
                | (df_customer["RFM_Score"] == 243) 
                    | (df_customer["RFM_Score"] == 242) 
                        | (df_customer["RFM_Score"] == 241) |
            (df_customer["RFM_Score"] == 144) 
                | (df_customer["RFM_Score"] == 143) 
                    | (df_customer["RFM_Score"] == 142) 
                        | (df_customer["RFM_Score"] == 141) |
            (df_customer["RFM_Score"] == 232) 
                | (df_customer["RFM_Score"] == 231)
                    | (df_customer["RFM_Score"] == 132) 
                        | (df_customer["RFM_Score"] == 131),
# "Inactive High Value" -------------------------------------------
            (df_customer["RFM_Score"] == 234) 
                | (df_customer["RFM_Score"] == 224) 
                    | (df_customer["RFM_Score"] == 214) 
                        | (df_customer["RFM_Score"] == 134) 
                            | (df_customer["RFM_Score"] == 124) |
            (df_customer["RFM_Score"] == 233)
                | (df_customer["RFM_Score"] == 223) 
                    | (df_customer["RFM_Score"] == 213)
                        | (df_customer["RFM_Score"] == 133)
                            | (df_customer["RFM_Score"] == 123), 
# "Inactive Customers" --------------------------------------------
            (df_customer["RFM_Score"] == 222) 
                | (df_customer["RFM_Score"] == 221)
                    | (df_customer["RFM_Score"] == 212)
                        | (df_customer["RFM_Score"] == 211),
# "Lost Customers" ------------------------------------------------
            (df_customer["RFM_Score"] == 122) 
                | (df_customer["RFM_Score"] == 121)
                    | (df_customer["RFM_Score"] == 114) 
                        | (df_customer["RFM_Score"] == 113) 
                            | (df_customer["RFM_Score"] == 112)
# "Lost Cheap Customers" ------------------------------------------
                | (df_customer["RFM_Score"] == 111)
]

In [29]:
mappings = ["01 Premium Customers", 
            "02 Loyal Customers", 
            "03 Frequent Buyers", 
            "04 Big Spenders", 
            "05 Promising New Cust", 
            "06 Inactive Frequent",
            "07 Inactive High Value",
            "08 Cheap Customers",
            "09 Lost Customers"]

df_customer["RFM_Segment"] = np.select(conditions, mappings, default="09-Lost Customers")
df_customer["RFM_Segment"].value_counts()

05 Promising New Cust     15
02 Loyal Customers        13
09 Lost Customers         13
07 Inactive High Value    11
06 Inactive Frequent      11
08 Cheap Customers         9
03 Frequent Buyers         7
01 Premium Customers       7
04 Big Spenders            5
Name: RFM_Segment, dtype: int64

***
# **Promotion**
***
Promotion strategy based on Labels

In [30]:
promotions = ["01 New Product & Loyalty programs", 
              "02 Priced Bundling & Loyalty programs",
              "03 Mixed Bundling & Discount Sale",
              "04 New Product & Priced Bundling",
              "05 Point Rewards & Discount Sale",
              "06 Returning Offers & Discount Sale",
              "07 eturning Offers & New Product",
              "08 Login Offers & Agressive Discount",
              "09 Don't bother to reaquire"]

df_customer["Promotion"] = np.select(conditions, promotions, default="09 Don't bother to reaquire")
df_customer["Promotion"].value_counts()

05 Point Rewards & Discount Sale         15
02 Priced Bundling & Loyalty programs    13
09 Don't bother to reaquire              13
07 eturning Offers & New Product         11
06 Returning Offers & Discount Sale      11
08 Login Offers & Agressive Discount      9
03 Mixed Bundling & Discount Sale         7
01 New Product & Loyalty programs         7
04 New Product & Priced Bundling          5
Name: Promotion, dtype: int64

***
# **Segments**
***
Display segmentations

In [36]:
df_customer.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,RFM_Segment,Promotion
CustomerId,,,,,,,,,
ALFKI,7,200,5317193.08,4,4,4,444,01 Premium Customers,01 New Product & Loyalty programs
ANATR,7,192,5208686.42,4,3,3,433,02 Loyal Customers,02 Priced Bundling & Loyalty programs
ANTON,6,191,5262405.09,4,3,3,433,02 Loyal Customers,02 Priced Bundling & Loyalty programs
AROUT,15,162,4395220.43,2,1,1,211,08 Cheap Customers,08 Login Offers & Agressive Discount
BERGS,6,206,5675614.51,4,4,4,444,01 Premium Customers,01 New Product & Loyalty programs


In [37]:
#pd.set_option('display.max_rows', None)
df_RFMScore = pd.pivot_table(df_customer, 
                             index=["CustomerId","RFM_Segment"], 
                             values=["RFM_Score"], 
                             columns=["Promotion"], 
                             fill_value=0)
df_RFMScore.head(25).sort_values("RFM_Segment")

RFM_Score  \
Promotion                         01 New Product & Loyalty programs   
CustomerId RFM_Segment                                                
ALFKI      01 Premium Customers                                 444   
BERGS      01 Premium Customers                                 444   
ANATR      02 Loyal Customers                                     0   
ANTON      02 Loyal Customers                                     0   
FAMIA      02 Loyal Customers                                     0   
ERNSH      02 Loyal Customers                                     0   
CHOPS      02 Loyal Customers                                     0   
FRANK      03 Frequent Buyers                                     0   
FOLIG      04 Big Spenders                                        0   
BOTTM      05 Promising New Cust                                  0   
CACTU      05 Promising New Cust                                  0   
BLONP      05 Promising New Cust                                  0   
FISSA      05 Promising New Cust                                  0   
BLAUS      05 Promising New Cust                                  0   
BSBEV      06 Inactive Frequent                                   0   
DRACD      07 Inactive High Value                                 0   
CONSH      07 Inactive High Value                                 0   
EASTC      07 Inactive High Value                                 0   
AROUT      08 Cheap Customers                                     0   
BONAP      08 Cheap Customers                                     0   
COMMI      08 Cheap Customers                                     0   
FOLKO      08 Cheap Customers                                     0   
BOLID      09 Lost Customers                                      0   
DUMON      09 Lost Customers                                      0   
CENTC      09 Lost Customers                                      0   

                                                                         \
Promotion                         02 Priced Bundling & Loyalty programs   
CustomerId RFM_Segment                                                    
ALFKI      01 Premium Customers                                       0   
BERGS      01 Premium Customers                                       0   
ANATR      02 Loyal Customers                                       433   
ANTON      02 Loyal Customers                                       433   
FAMIA      02 Loyal Customers                                       443   
ERNSH      02 Loyal Customers                                       443   
CHOPS      02 Loyal Customers                                       433   
FRANK      03 Frequent Buyers                                         0   
FOLIG      04 Big Spenders                                            0   
BOTTM      05 Promising New Cust                                      0   
CACTU      05 Promising New Cust                                      0   
BLONP      05 Promising New Cust                                      0   
FISSA      05 Promising New Cust                                      0   
BLAUS      05 Promising New Cust                                      0   
BSBEV      06 Inactive Frequent                                       0   
DRACD      07 Inactive High Value                                     0   
CONSH      07 Inactive High Value                                     0   
EASTC      07 Inactive High Value                                     0   
AROUT      08 Cheap Customers                                         0   
BONAP      08 Cheap Customers                                         0   
COMMI      08 Cheap Customers                                         0   
FOLKO      08 Cheap Customers                                         0   
BOLID      09 Lost Customers                                          0   
DUMON      09 Lost Customers                                          0   
CENTC      09 Lost Customers                